# Imports

In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import plotly.graph_objs as go
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from utils import TrainTestSequenceDataset, smape, process_for_train, features, targets, spherical_from_cartesian
from models import LSTM, ResNet18
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import ipywidgets as widgets
import os
from spaceopt import SpaceOpt

# Loading data

In [ ]:
data = pd.read_csv('train.csv', parse_dates=['epoch'])
test_sat_id = torch.load('test_sat_id')

# Data processing

In [ ]:
sat_datas_train, sat_datas_test = process_for_train(data)

In [ ]:
sat_datas_train[0].head()

# Model

In [ ]:
seq_len = 20
hidden_dim = 100
num_layers = 2
model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers)
model.train()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.02, eps=1e-2)
criterion = smape
n = 10

# Train on n satellite

In [ ]:
model.train()
loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
display(loss_widget)
train_data = sat_datas_train[n]
x_train = train_data[features]
y_train = train_data[targets]
train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
for epoch in tqdm(range(15)):
    for seq_train_x, seq_train_y in train_dataloader:
        model.zero_grad()  # refresh gradients
        model.init_hidden_cell()
        predictions = model(seq_train_x)
        loss = criterion(predictions, seq_train_y)
        loss_widget.value = loss.mean()
        loss.mean().backward()  # compute gradients
        optimizer.step()  # update network parameters

In [ ]:
model.eval()
score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
display(score_widget)
test_data = sat_datas_test[n]
x_test = test_data[features]
y_test = test_data[targets]
test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
loss_sum = 0
i = 0

for seq_test_x, seq_test_y in tqdm(test_dataloader):
    with torch.no_grad():
        model.init_hidden_cell()
        predictions = model(seq_test_x)
        loss = criterion(predictions, seq_test_y).mean()
        loss_sum += loss
        i += 1
        score_widget.value = 1 - loss_sum / i
score =  (1 - loss_sum / i).item()
print(score)

In [ ]:
search_space = {
    'lr': [0.001, 0.01, 0.1, 1.],
    'eps': [1e-8, 1e-4, 1e-2, 1e0],
    'seq_len': [10, 20, 30, 40],
    'hidden_dim': [20, 30, 50],
    'epoch': [10],
    'num_layers': [1, 2, 3, 4]
}

In [ ]:
def evaluate_new(spoint, sat_id):
    seq_len = spoint['seq_len']
    hidden_dim = spoint['hidden_dim']
    num_layers = spoint['num_layers']
    ep = spoint['epoch']
    lr = spoint['lr']
    eps = spoint['eps']
    
    train_data = sat_datas_train[sat_id]
    if seq_len > int(len(train_data) / 2 - 1):
        seq_len = int(len(train_data) / 2 - 1)
    x_train = train_data[features]
    y_train = train_data[targets]
    
    model = LSTM(hidden_dim=hidden_dim, seq_len=seq_len, num_layers=num_layers)
    model.train()
    optimizer = torch.optim.Adam(params=model.parameters(), lr=lr, eps=eps)
    criterion = smape
    model.train()
    loss_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Loss', value=0)  # jupyter widget
    display(loss_widget)
    train_data = sat_datas_train[n]
    x_train = train_data[features]
    y_train = train_data[targets]
    train_dataset = TrainTestSequenceDataset(x_train, y_train, seq_len=model.seq_len)
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
    for epoch in tqdm(range(ep)):
        for seq_train_x, seq_train_y in train_dataloader:
            model.zero_grad()  # refresh gradients
            model.init_hidden_cell()
            predictions = model(seq_train_x)
            loss = criterion(predictions, seq_train_y)
            loss_widget.value = loss.mean()
            loss.mean().backward()  # compute gradients
            optimizer.step()  # update network parameters

    model.eval()
    score_widget = widgets.FloatProgress(min=0, max=1, step=0.01, description='Score', value=0)  # jupyter widget
    display(score_widget)
    test_data = sat_datas_test[n]
    x_test = test_data[features]
    y_test = test_data[targets]
    test_dataset = TrainTestSequenceDataset(x_test, y_test, seq_len=model.seq_len)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)
    loss_sum = 0
    i = 0

    for seq_test_x, seq_test_y in tqdm(test_dataloader):
        with torch.no_grad():
            model.init_hidden_cell()
            predictions = model(seq_test_x)
            loss = criterion(predictions, seq_test_y).mean()
            loss_sum += loss
            i += 1
            score_widget.value = 1 - loss_sum / i
    score =  (1 - loss_sum / i).item()

    return score, model

In [ ]:
best_params = {'lr': 0.02, 'eps': 1e-2, 'seq_len': 20,
               'hidden_dim': 100, 'epoch': 30, 'num_layers': 2}

In [ ]:
results = {}
for sat_id in test_sat_id:
    clear_output()
    print(f'Satellite id: {sat_id}')
    print(results)
    spoint = best_params
    
    score, model= evaluate_new(spoint=spoint, sat_id=sat_id)
    spoint['score'] = score
    if not os.path.exists(f'models//{sat_id}'):
        os.makedirs(f'models//{sat_id}')
    results[str(sat_id)] = score
    torch.save(score, f'models//{sat_id}//score')
    torch.save(model, f'models//{sat_id}//model.pt')